In [ ]:
!pip install firebase

In [ ]:
# ------------------------------------------ JavaScript Container ------------------------------------------ #

app_js = """
document.addEventListener('DOMContentLoaded', function() {
    if (window.location.pathname.endsWith('TeamGraph.html')) {
        const userActionMetrics = JSON.parse(localStorage.getItem('userActionMetrics'));
        if (userActionMetrics) {
            displayTotalActionsChart(userActionMetrics);
            displayActionDistributionChart(userActionMetrics);
            displayActionsPerDayChart(userActionMetrics);
        }
    } else if (window.location.pathname.endsWith('notifications.html')) {
        displayNotifications();
    } else if (window.location.pathname.endsWith('glossary.html')) {
        const savedJson = localStorage.getItem('uploadedJson');
        if (savedJson) {
            const jsonData = JSON.parse(savedJson);
            parseGlossary(jsonData);
        }
    } else {
        document.getElementById('jsonUploadForm').addEventListener('submit', function(event) {
            event.preventDefault(); // Prevent the form from submitting via the browser
            uploadJson();
        });
    }
});

/* function uploadJson() {
    const fileInput = document.getElementById('fileInput');
    if (fileInput.files.length > 0) {
        const reader = new FileReader();
        reader.onload = function(event) {
            try {
                const jsonObj = JSON.parse(event.target.result);
                console.log('JSON Object:', jsonObj);
                localStorage.setItem('uploadedJson', JSON.stringify(jsonObj));
                parseJson(jsonObj);
                alert('JSON file uploaded and processed.');
                if (window.location.pathname.endsWith('notifications.html')) {
                    displayNotifications();
                }
            } catch (e) {
                alert('Invalid JSON file');
            }
        };
        reader.readAsText(fileInput.files[0]);
    } else {
        alert('Please select a file');
    }
} */



function parseJson(jsonObj) {
    const actions = ["Add", "Cancel", "Close", "Commit", "Copy", "Delete", "Drag", "Edit", "Insert"];
    const userActionMetrics = {};

    jsonObj.forEach(entry => {
        const user = entry.User;
        const description = entry.Description;

        if (!userActionMetrics[user]) {
            userActionMetrics[user] = { totalActions: 0, actionTypes: {}, actionsPerDay: {} };
            actions.forEach(action => userActionMetrics[user].actionTypes[action] = 0);
        }

        userActionMetrics[user].totalActions++;

        actions.forEach(action => {
            if (description.includes(action)) {
                userActionMetrics[user].actionTypes[action]++;
            }
        });

        const date = new Date(entry.Time).toDateString();
        if (!userActionMetrics[user].actionsPerDay[date]) {
            userActionMetrics[user].actionsPerDay[date] = 0;
        }
        userActionMetrics[user].actionsPerDay[date]++;
    });

    localStorage.setItem('userActionMetrics', JSON.stringify(userActionMetrics));
}

function displayTotalActionsChart(metrics) {
    const ctx = document.getElementById('totalActionsChart').getContext('2d');
    const labels = Object.keys(metrics);
    const data = labels.map(user => metrics[user].totalActions);

    new Chart(ctx, {
        type: 'bar',
        data: {
            labels,
            datasets: [{
                label: 'Total Actions',
                data,
                backgroundColor: 'rgba(75, 192, 192, 0.2)',
                borderColor: 'rgba(75, 192, 192, 1)',
                borderWidth: 1
            }]
        },
        options: {
            scales: {
                y: { beginAtZero: true }
            }
        }
    });
}

function displayActionDistributionChart(metrics) {
    const ctx = document.getElementById('actionDistributionChart').getContext('2d');
    const labels = Object.keys(metrics);
    const actionTypes = Object.keys(metrics[labels[0]].actionTypes);

    const datasets = actionTypes.map(action => ({
        label: action,
        data: labels.map(user => metrics[user].actionTypes[action]),
        backgroundColor: getRandomColor(),
        borderColor: 'rgba(0, 0, 0, 0.1)',
        borderWidth: 1
    }));

    new Chart(ctx, {
        type: 'bar',
        data: {
            labels,
            datasets
        },
        options: {
            scales: {
                y: { beginAtZero: true },
                x: { stacked: true },
                y: { stacked: true }
            }
        }
    });
}

function displayActionsPerDayChart(metrics) {
    const ctx = document.getElementById('actionsPerDayChart').getContext('2d');
    const labels = Object.keys(metrics);
    const allDates = [];

    labels.forEach(user => {
        Object.keys(metrics[user].actionsPerDay).forEach(date => {
            if (!allDates.includes(date)) {
                allDates.push(date);
            }
        });
    });

    const datasets = labels.map(user => ({
        label: user,
        data: allDates.map(date => metrics[user].actionsPerDay[date] || 0),
        fill: false,
        borderColor: getRandomColor(),
        tension: 0.1
    }));

    new Chart(ctx, {
        type: 'line',
        data: {
            labels: allDates,
            datasets
        },
        options: {
            scales: {
                y: { beginAtZero: true }
            }
        }
    });
}

function getRandomColor() {
    const r = Math.floor(Math.random() * 255);
    const g = Math.floor(Math.random() * 255);
    const b = Math.floor(Math.random() * 255);
    return `rgba(${r}, ${g}, ${b}, 0.5)`;
}

// Notifications related functions
function displayNotifications() {
    const savedJson = localStorage.getItem('uploadedJson');
    if (!savedJson) {
        alert('No notifications available. Please upload a JSON file.');
        return;
    }

    const jsonData = JSON.parse(savedJson);
    const notifications = jsonData.filter(entry => entry.Description.toLowerCase().includes('notification'));
    const notificationList = document.getElementById('notificationList');
    const notificationsPerPage = 5;
    let currentPage = 1;

    function renderNotifications(filteredNotifications) {
        notificationList.innerHTML = '';
        const startIndex = (currentPage - 1) * notificationsPerPage;
        const endIndex = startIndex + notificationsPerPage;
        const currentNotifications = filteredNotifications.slice(startIndex, endIndex);

        currentNotifications.forEach(notification => {
            const notificationItem = document.createElement('div');
            notificationItem.className = 'notification-item' + (notification.read ? ' read' : ' unread');
            notificationItem.innerHTML = `
                <h3>${notification.type}</h3>
                <p>${notification.Description}</p>
                <span>${notification.Time}</span>
                <button onclick="markAsRead(${notification.id})">Mark as Read</button>
                <button onclick="deleteNotification(${notification.id})">Delete</button>
            `;
            notificationList.appendChild(notificationItem);
        });

        updatePageInfo(filteredNotifications);
    }

    function updatePageInfo(filteredNotifications) {
        const pageInfo = document.getElementById('pageInfo');
        const totalPages = Math.ceil(filteredNotifications.length / notificationsPerPage);
        pageInfo.textContent = `Page ${currentPage} of ${totalPages}`;
    }

    window.markAsRead = function(id) {
        const notification = notifications.find(n => n.id === id);
        if (notification) {
            notification.read = true;
            renderNotifications(notifications);
        }
    };

    window.deleteNotification = function(id) {
        const index = notifications.findIndex(n => n.id === id);
        if (index !== -1) {
            notifications.splice(index, 1);
            renderNotifications(notifications);
        }
    };

    window.prevPage = function() {
        if (currentPage > 1) {
            currentPage--;
            renderNotifications(notifications);
        }
    };

    window.nextPage = function() {
        const totalPages = Math.ceil(notifications.length / notificationsPerPage);
        if (currentPage < totalPages) {
            currentPage++;
            renderNotifications(notifications);
        }
    };

    function filterNotifications() {
        const searchValue = document.getElementById('searchNotifications').value.toLowerCase();
        const filterType = document.getElementById('filterType').value;

        const filteredNotifications = notifications.filter(notification => {
            const matchesSearch = notification.Description.toLowerCase().includes(searchValue);
            const matchesType = filterType === '' || notification.type === filterType;
            return matchesSearch && matchesType;
        });

        renderNotifications(filteredNotifications);
    }

    document.getElementById('searchNotifications').addEventListener('keyup', filterNotifications);
    document.getElementById('filterType').addEventListener('change', filterNotifications);

    renderNotifications(notifications);
}

// Glossary related functions
function parseGlossary(jsonData) {
    const actions = ["Add", "Cancel", "Close", "Commit", "Copy", "Delete", "Drag", "Edit", "Insert"];
    const userActionCount = {};

    jsonData.forEach(entry => {
        const user = entry.User;
        const description = entry.Description;

        actions.forEach(action => {
            if (description.includes(action)) {
                if (!userActionCount[action]) {
                    userActionCount[action] = {};
                }
                if (!userActionCount[action][user]) {
                    userActionCount[action][user] = 0;
                }
                userActionCount[action][user]++;
            }
        });
    });

    const glossaryTable = document.getElementById('glossaryTable');
    glossaryTable.innerHTML = ''; // Clear default values

    const descriptions = {
        "Add": "Introduce a new element or feature. - Example: \"Add part studio feature.\"",
        "Cancel": "Abort an ongoing operation. - Example: \"Cancel Operation.\"",
        "Close": "Exit a document or tab. - Example: \"Tab Part Studio 3 of type PARTSTUDIO closed.\"",
        "Commit": "Confirm the addition or modification of a feature. - Example: \"Commit add or edit of part studio feature.\"",
        "Copy": "Duplicate an element or feature. - Example: \"Copy paste sketch.\"",
        "Delete": "Remove an existing element or feature. - Example: \"Delete part studio feature.\"",
        "Drag": "Move an element by clicking and holding it. - Example: \"Drag: instance.\"",
        "Edit": "Modify an existing element or feature. - Example: \"Edit: Sketch 1.\"",
        "Insert": "Place a new feature at a specified location."
    };

    for (const action in userActionCount) {
        const row = document.createElement('tr');
        row.innerHTML = `<td>${action}</td><td>${descriptions[action]}</td><td><details><summary>Details</summary><ul id="${action}List"></ul></details></td>`;
        glossaryTable.appendChild(row);

        const actionList = document.getElementById(`${action}List`);
        const sortedUsers = Object.entries(userActionCount[action]).sort((a, b) => b[1] - a[1]);
        sortedUsers.forEach(([user, count]) => {
            const listItem = document.createElement('li');
            listItem.textContent = `${user.padEnd(10)}: ${count.toString().padStart(4)}`;
            actionList.appendChild(listItem);
        });
    }
}


"""

In [ ]:
style_css = """
/* Add button styling */
.button-container {
    display: flex;
    justify-content: center;
    margin-bottom: 20px;
    flex-wrap: wrap;
    position: fixed;
    top: 0;
    width: 100%;
    background-color: #2c3e50;
    z-index: 1000;
    padding: 10px 0;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

.nav-button {
    padding: 10px 20px;
    margin: 10px;
    background-color: #3498db; /* Primary color */
    color: white; /* White text */
    border: none;
    border-radius: 5px;
    text-decoration: none;
    font-size: 16px;
    cursor: pointer;
    transition: background-color 0.3s ease, transform 0.3s ease;
    text-align: center;
    max-width: 200px; /* Ensure buttons don't get too large */
}

.nav-button:hover {
    background-color: #2980b9;
    transform: translateY(-2px);
}

.nav-button:disabled {
    background-color: gray; /* Gray background when disabled */
    cursor: not-allowed;
}

/* Responsive layout */
@media (max-width: 768px) {
    .nav-button {
        font-size: 14px;
        padding: 8px 16px;
        max-width: 150px; /* Adjust button size for smaller screens */
    }

    .button-container {
        flex-direction: column;
        align-items: center;
    }
}

body {
    margin: 0;
    padding-top: 60px; /* Adjust based on the height of the fixed button container */
    background: linear-gradient(to right, #77a0bc, #7eb896);
    color: #465b70;
    font-family: 'Arial', sans-serif;
    box-sizing: border-box;
}

.container {
    display: flex;
    flex-direction: column;
    align-items: center;
    padding: 20px;
    min-height: 100vh;
    justify-content: center;
}

form {
    width: 100%;
    max-width: 500px;
    padding: 20px;
    background-color: #fff;
    border-radius: 10px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    box-sizing: border-box;
}

form input[type="file"] {
    width: 100%;
    padding: 10px;
    margin: 10px 0;
    border: 1px solid #ddd;
    border-radius: 5px;
}

form button {
    width: 100%;
    padding: 10px;
    background-color: #3498db;
    border: none;
    border-radius: 5px;
    color: white;
    font-size: 16px;
    cursor: pointer;
    transition: background-color 0.3s ease;
}

form button:hover {
    background-color: #2980b9;
}

h2, h1 {
    text-align: center;
    color: #fff;
    margin-bottom: 20px;
}

h3{
  text-align: center;
  color: #000000;
  font-size: 36px;
  margin-bottom: 10px;
}


ul {
    list-style: none;
    padding: 0;
    margin: 0;
    text-align: center;
}

th, td {
  padding: 10px;
  text-align: left;
  border: 1px solid #ddd;
}
th {
  background-color: #f2f2f2;
}
tr:nth-child(even) {
  background-color: #f9f9f9;
}
tr:hover {
  background-color: #f1f1f1;
}


/* Enhanced table styling */
table {
    width: 100%;
    max-width: 800px;
    margin: 20px auto;
    border-collapse: collapse;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    background: linear-gradient(to left, #4cabeb, #4ce68c);
    background-color: #fff;
}

th, td {
    padding: 12px 15px;
    text-align: left;
    border: 1px solid #ddd;
    vertical-align: top;
}

details {
    padding: 10px;
    width: 200px; /* Adjust width to accommodate name and count */
}

details ul {
    padding-left: 20px;
    text-align: left;
}

/* Custom styles for the summary */
summary {
    font-weight: bold;
    cursor: pointer;
}

details[open] summary {
    background-color: #e0e0e0;
}

details[open] summary + * {
    margin-top: 10px;
}

details ul {
    padding: 0;
    margin: 0;
}

details li {
    padding: 5px 0;
    list-style: none;
    font-family: monospace; /* Use a monospace font for better alignment */
    white-space: pre; /* Preserve spaces for alignment */
}

/* Chart container styling */
.chart-container {
    width: 100%;
    max-width: 600px; /* Set maximum width for charts */
    margin: 20px auto; /* Add margin for spacing between charts */
    padding: 20px; /* Add padding for better alignment */
    background-color: #fff; /* Set background color */
    border-radius: 10px; /* Add border radius for rounded corners */
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); /* Add shadow for better visual */
}

/* Notification styling */
.notification-item {
    background-color: #fff;
    padding: 15px;
    margin: 10px 0;
    border-radius: 5px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    width: 100%;
    max-width: 600px;
    display: flex;
    flex-direction: column;
}

.notification-item.read {
    background-color: #e0e0e0;
}

.notification-item h3 {
    margin: 0;
    font-size: 18px;
}

.notification-item p {
    margin: 5px 0;
}

.notification-item span {
    font-size: 12px;
    color: #999;
}

.notification-actions {
    display: flex;
    justify-content: space-between;
    margin-top: 10px;
}

.notification-actions button {
    padding: 5px 10px;
    background-color: #3498db;
    border: none;
    border-radius: 5px;
    color: white;
    font-size: 14px;
    cursor: pointer;
    transition: background-color 0.3s ease;
}

.notification-actions button:hover {
    background-color: #2980b9;
}

.notifications-header {
    width: 100%;
    max-width: 600px;
    display: flex;
    justify-content: space-between;
    margin-bottom: 20px;
}

.notifications-header input {
    width: 80%;
    padding: 10px;
    border: 1px solid #ddd;
    border-radius: 5px;
}

.notifications-header button {
    width: 18%;
    padding: 10px;
    background-color: #3498db;
    border: none;
    border-radius: 5px;
    color: white;
    font-size: 16px;
    cursor: pointer;
    transition: background-color 0.3s ease;
}

.notifications-header button:hover {
    background-color: #2980b9;
}

.pagination {
    display: flex;
    justify-content: center;
    align-items: center;
    margin-top: 20px;
}

.pagination button {
    padding: 10px 20px;
    margin: 0 10px;
    background-color: #3498db;
    border: none;
    border-radius: 5px;
    color: white;
    font-size: 16px;
    cursor: pointer;
    transition: background-color 0.3s ease;
}

.pagination button:hover {
    background-color: #2980b9;
}

.pagination span {
    font-size: 16px;
}

.info {
    font-size: 0.9em;
    color: #555;
    display: none;
}
.more-info {
    cursor: pointer;
    color: blue;
    text-decoration: underline;
}

.modal {
  display: none;
  position: fixed;
  z-index: 1;
  padding-top: 60px;
  left: 0;
  top: 0;
  width: 100%;
  height: 100%;
  overflow: auto;
  background-color: rgb(0,0,0);
  background-color: rgba(0,0,0,0.4);
}

.modal-content {
  background-color: #fefefe;
  margin: 5% auto;
  padding: 20px;
  border: 1px solid #888;
  width: 80%;
}

.close {
  color: #aaa;
  float: right;
  font-size: 28px;
  font-weight: bold;
}

.close:hover,
.close:focus {
  color: black;
  text-decoration: none;
  cursor: pointer;
}


"""

In [ ]:
uploadjson = """
        function uploadJson() {
            const fileInput = document.getElementById('fileInput');
            const file = fileInput.files[0];

            if (file) {
                const reader = new FileReader();
                reader.onload = function(event) {
                    const content = event.target.result;
                    google.colab.kernel.invokeFunction('notebook.uploadJSON', [file.name, content], {});
                };
                reader.readAsText(file);
            } else {
                alert('No file selected');
            }
        }
        function loadPage(page) {
    const contentDiv = document.getElementById('contentDiv');
    console.log('Loading page:', page); // Debugging log

    if (page === 'JSONUpload') {
        google.colab.kernel.invokeFunction('notebook.moveUpload', [], {});
    } else if (page === 'Glossary') {
        google.colab.kernel.invokeFunction('notebook.moveGlossary', [], {});
    } else if (page === 'TeamStats') {
        google.colab.kernel.invokeFunction('notebook.moveTeamStats', [], {});
    } else if (page === 'Notifications') {
        google.colab.kernel.invokeFunction('notebook.moveNotifications', [], {});
    } else {
        contentDiv.innerHTML = `<h1>Notifications</h1>
        <div class="notifications-header">
            <select id="filterType" onchange="filterNotifications()">
                <option value="">All Types</option>
                <option value="System Alert">System Alert</option>
                <option value="User Activity">User Activity</option>
            </select>
            <input type="text" id="searchNotifications" placeholder="Search notifications..." onkeyup="filterNotifications()">
        </div>
        <div id="notificationList">
            <!-- Notifications will be dynamically inserted here -->
        </div>
        <div class="pagination">
            <button onclick="prevPage()">Previous</button>
            <span id="pageInfo"></span>
            <button onclick="nextPage()">Next</button>
        </div>`;
        console.log('Calling displayNotifications');
        displayNotifications();  // Call the display function immediately
    }
}

// Ensure the loadPage function is called with 'Notifications' to test the notifications display
document.addEventListener('DOMContentLoaded', function() {
    loadPage('Notifications');
});
    """

In [ ]:
from firebase import firebase
def upload_cloud(jsonData):
  FBconn = firebase.FirebaseApplication('https://cloudtut-4b319-default-rtdb.firebaseio.com/',None)
  result = FBconn.post('/myTest2/',jsonData)
  print(result)

In [ ]:
import re

def index_words_from_json(json_data):
    index = {}

    def index_string(s):
        words = re.findall(r'\w+', s)
        for word in words:
            word = word.lower()
            if word in index:
                index[word] += 1
            else:
                index[word] = 1

    def parse_json(data):
        if isinstance(data, dict):
            for value in data.values():
                parse_json(value)
        elif isinstance(data, list):
            for item in data:
                parse_json(item)
        elif isinstance(data, str):
            index_string(data)

    parse_json(json_data)
    return index

In [ ]:
def remove_stop_words(index):
    stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to'}
    for stop_word in stop_words:
        if stop_word in index:
            del index[stop_word]
    return index

In [ ]:
from nltk.stem import PorterStemmer

def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count
    return stemmed_index

In [ ]:
def search(query, index):

    stemmer = PorterStemmer()
    query_words = re.findall(r'\w+', query.lower())
    print(query)
    results = {}
    for word in query_words:
        word = stemmer.stem(word)
        if word in index:
            results[word] = index[word]
    return results

In [ ]:
def search_engine(url, query):
    soup = url
    if soup is None:
        return None
    index = index_words_from_json(soup)
    index = remove_stop_words(index)
    index = apply_stemming(index)
    results = search(query, index)
    return results

In [ ]:

def searchJson(data,query,location):
  results = search_engine(data, query)
  print(results)
  rank=1
  for word, count in results.items():
    rank = rank*1/count
  rank = 1-rank
  print(rank)
  FBconn = firebase.FirebaseApplication('https://cloudtut-4b319-default-rtdb.firebaseio.com/',None)
  result = FBconn.put(location, 'results', results)
  print(result)
  return result


In [ ]:
uploadPage="""
    <h2>Upload JSON File</h2>
    <form id="jsonUploadForm">
        <input type="file" id="fileInput" accept=".json">
        <button type="button" onclick="uploadJson()">Upload</button>
    </form>
"""

glossaryPage="""
<div id="glossaryContainer">
            <h2>Glossary</h2>
            <ul id="glossaryList"></ul>
        </div>
        <table>
            <tr>
                <th>Word</th>
                <th>Description</th>
                <th>Details</th>
            </tr>
            <tbody id="glossaryTable">
                <!-- Default empty values -->
                <tr>
                    <td>Add</td>
                    <td>Introduce a new element or feature. - Example: "Add part studio feature."</td>
                    <td><details><summary>Details</summary><ul></ul></details></td>
                </tr>
                <tr>
                    <td>Cancel</td>
                    <td>Abort an ongoing operation. - Example: "Cancel Operation."</td>
                    <td><details><summary>Details</summary><ul></ul></details></td>
                </tr>
                <tr>
                    <td>Close</td>
                    <td>Exit a document or tab. - Example: "Tab Part Studio 3 of type PARTSTUDIO closed."</td>
                    <td><details><summary>Details</summary><ul></ul></details></td>
                </tr>
                <tr>
                    <td>Commit</td>
                    <td>Confirm the addition or modification of a feature. - Example: "Commit add or edit of part studio feature."</td>
                    <td><details><summary>Details</summary><ul></ul></details></td>
                </tr>
                <tr>
                    <td>Copy</td>
                    <td>Duplicate an element or feature. - Example: "Copy paste sketch."</td>
                    <td><details><summary>Details</summary><ul></ul></details></td>
                </tr>
                <tr>
                    <td>Delete</td>
                    <td>Remove an existing element or feature. - Example: "Delete part studio feature."</td>
                    <td><details><summary>Details</summary><ul></ul></details></td>
                </tr>
                <tr>
                    <td>Drag</td>
                    <td>Move an element by clicking and holding it. - Example: "Drag: instance."</td>
                    <td><details><summary>Details</summary><ul></ul></details></td>
                </tr>
                <tr>
                    <td>Edit</td>
                    <td>Modify an existing element or feature. - Example: "Edit: Sketch 1."</td>
                    <td><details><summary>Details</summary><ul></ul></details></td>
                </tr>
                <tr>
                    <td>Insert</td>
                    <td>Place a new feature at a specified location.</td>
                    <td><details><summary>Details</summary><ul></ul></details></td>
                </tr>
            </tbody>
        </table>
"""

graphPage="""
  <h1>Team Stats</h1>
        <div class="chart-container">
            <h3 id="teamRanking"></h3>
        <span class="more-info" onclick="toggleInfo()">More Info</span>
        <div id="infoText" class="info">
            The team ranking is calculated based on performance metrics, Actions done by the team summed up and normalized.
        </div>
          </div>
        </div>
        <div class="chart-container">
            <table>
              <thead>
                <tr>
                  <th>Student</th>
                  <th>Score</th>
                </tr>
              </thead>
              <tbody id="leaderboard">
              </tbody>
            </table>
        </div>
        <div class="chart-container">
            <canvas id="myChart" width="400" height="200"></canvas>
        </div>
        <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
"""
notificationPage = """
<h1>Notifications</h1>
    <div id="notificationList">
        <!-- Notifications will be dynamically inserted here -->
    </div>
    <div id="notificationModal" class="modal">
        <div class="modal-content">
            <span class="close">&times;</span>
            <p id="modalContent"></p>
        </div>
    </div>
"""

notificationPageJs = """
document.addEventListener('DOMContentLoaded', function() {
    displayNotifications();
});

function displayNotifications() {
    const latestNotifications = JSON.parse(localStorage.getItem('latestNotifications')) || [];
    const notificationList = document.getElementById('notificationList');
    const modal = document.getElementById("notificationModal");
    const span = document.getElementsByClassName("close")[0];

    console.log('Latest Notifications:', latestNotifications); // Debugging log

    notificationList.innerHTML = '';

    if (latestNotifications.length === 0) {
        notificationList.innerHTML = '<p>No notifications available.</p>';
        return;
    }

    latestNotifications.forEach(notification => {
        const notificationItem = document.createElement('div');
        notificationItem.className = 'notification-item';
        notificationItem.innerHTML = `
            <p><strong>${notification.student_name}</strong>: ${notification.description}</p>
            <span>${new Date(notification.timestamp * 1000).toLocaleString()}</span>
        `;
        notificationItem.onclick = function() {
            const modalContent = document.getElementById("modalContent");
            modalContent.innerText = `Student: ${notification.student_name}\nDescription: ${notification.description}`;
            modal.style.display = "block";
        };
        notificationList.appendChild(notificationItem);
    });

    // Close the modal when the user clicks on <span> (x)
    span.onclick = function() {
        modal.style.display = "none";
    }

    // Close the modal when the user clicks anywhere outside of the modal
    window.onclick = function(event) {
        if (event.target == modal) {
            modal.style.display = "none";
        }
    }
}
"""

In [ ]:
def calculate_actions(actions):
    positive_actions = ['add', 'insert', 'commit', 'chang', 'edit', 'updat', 'fix', 'start']
    negative_actions = ['delet', 'cancel']

    positive_sum = sum(actions[action] for action in positive_actions if action in actions)
    negative_sum = -sum(actions[action] for action in negative_actions if action in actions)
    total_sum = positive_sum + negative_sum

    max_sum = positive_sum
    min_sum = negative_sum

    normalized_total_sum = (total_sum - min_sum) / (max_sum - min_sum)
    normalized_total_sum = normalized_total_sum * 10
    return f"{normalized_total_sum:.1f}"




In [ ]:

glossaryPageJs=""""""
uploadPageJs=""""""

In [ ]:
def generate_combined_js(leaderboard_data, chart_data, heading_text):
    # Convert the data dictionaries to JavaScript object notation
    js_leaderboard_data = str(leaderboard_data).replace("'", '"')
    js_chart_data = str(chart_data).replace("'", '"')
    js_heading_text = heading_text.replace("'", "\\'")

    # Combined JavaScript template with placeholders for data and heading text
    js_template = """
    function showLeaderboard(data) {
        const leaderboard = document.getElementById('leaderboard');
        // Convert data to an array of [key, value] pairs and sort by the value in descending order
        const sortedData = Object.entries(data).sort((a, b) => b[1] - a[1]);

        // Clear existing rows
        leaderboard.innerHTML = '';

        // Append sorted data to the leaderboard
        sortedData.forEach(([student, score]) => {
            const row = document.createElement('tr');
            const studentCell = document.createElement('td');
            const scoreCell = document.createElement('td');

            studentCell.textContent = student;
            scoreCell.textContent = score;

            row.appendChild(studentCell);
            row.appendChild(scoreCell);
            leaderboard.appendChild(row);
        });
    }

    function createBarChart(data) {
        // Extract labels and values from the data object
        const labels = Object.keys(data);
        const values = Object.values(data);

        // Get the context of the canvas element we want to select
        const ctx = document.getElementById('myChart').getContext('2d');

        // Create the bar chart
        new Chart(ctx, {
            type: 'bar',
            data: {
                labels: labels,
                datasets: [{
                    label: 'Actions Made',
                    data: values,
                    backgroundColor: 'rgba(75, 192, 192, 0.2)',
                    borderColor: 'rgba(75, 192, 192, 1)',
                    borderWidth: 1
                }]
            },
            options: {
                scales: {
                    y: {
                        beginAtZero: true
                    }
                }
            }
        });
    }

    function showTeamRanking(headingText) {
        // Get the element to display the heading
        const heading = document.getElementById('teamRanking');

        // Set the heading text
        heading.textContent = 'Team overall Ranking: ' + headingText;
    }

    // Insert the data and heading text here
    const leaderboardData = %s;
    const chartData = %s;
    const headingText = '%s';

    // Show the leaderboard with the data
    showLeaderboard(leaderboardData);

    // Create the chart with the data
    createBarChart(chartData);

    // Show the team ranking with the heading text
    showTeamRanking(headingText);

    function toggleInfo() {
    var info = document.getElementById("infoText");
    if (info.style.display === "none" || info.style.display === "") {
        info.style.display = "block";
    } else {
        info.style.display = "none";
     }
    }

    """ % (js_leaderboard_data, js_chart_data, js_heading_text)

    return js_template




In [ ]:
from bs4 import BeautifulSoup

def mod_html(html_content, data):

    # Create BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all text nodes containing "Details"
    details_tags = soup.find_all(string='Details')

    # Replace each occurrence with a number from the array
    for index, detail in enumerate(details_tags):
        if index >= 1 and (index-1) < len(data):
            # Get the number to replace "Details" from the dictionary
            replacement_number = list(data.values())[index-1]
            detail.replace_with(str(replacement_number))

    # Return the modified HTML
    return soup.prettify()



In [ ]:
# prompt: get this from firebase https://cloudtut-4b319-default-rtdb.firebaseio.com/SearchResults/-O2ZwcBBjhphj4UFxdP1


firebase_url = 'https://cloudtut-4b319-default-rtdb.firebaseio.com/'
search_results_path = 'SearchResults'
student_results_path = 'StudentResults'

# Initialize Firebase connection
FBconn = firebase.FirebaseApplication(firebase_url, None)

# Retrieve all search results data
allSearchResults = FBconn.get(search_results_path, None)

# Get the first entry in the search results
firstSearchResultKey = list(allSearchResults.keys())[0]
actionData = allSearchResults[firstSearchResultKey]

# Retrieve all student results data
allStudentResults = FBconn.get(student_results_path, None)

# Get the first entry in the student results
firstStudentResultKey = list(allStudentResults.keys())[0]
studentData = allStudentResults[firstStudentResultKey]



In [ ]:
# prompt: a html variable that i can add an existing js and css variables into

from IPython.display import HTML
from IPython.display import clear_output
from google.colab import output
from IPython.display import display, HTML
import json
import os

def upload_json(filename, content):
    # Parse the JSON content
    data = json.loads(content)

    # Save the JSON data to a file in the current directory
    filepath = os.path.join(os.getcwd(), filename)
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=2)

    print(f"File '{filename}' has been saved successfully in the Colab folder.")

    # Print the first item from the JSON data
    if isinstance(data, dict):
        first_key = next(iter(data))
        print(f"First item in the JSON: {first_key}: {data[first_key]}")
    elif isinstance(data, list) and len(data) > 0:
        print(f"First item in the JSON: {data[0]}")
    else:
        print("The JSON file is empty or has an unexpected structure.")
    upload_cloud(data)
    actionData = searchJson(data,'Add Insert Commit Delete Change Edit Update Cancel Fix Start','/SearchResults/')
    studentData = searchJson(data,'StudentA StudentB StudentC','/StudentResults/')




jsonData = output.register_callback('notebook.uploadJSON', upload_json)




def move_upload():
  clear_output()
  displayHtml(uploadPage,uploadPageJs)
def move_glossary():
  clear_output()
  modGlossaryPage = mod_html(glossaryPage, actionData)
  displayHtml(modGlossaryPage,glossaryPageJs)
def move_team_stats():
  clear_output()
  graphPageJs = generate_combined_js(studentData, actionData,calculate_actions(actionData))
  displayHtml(graphPage,graphPageJs)
def move_notifications():
    clear_output()
    displayHtml(notificationPage, notificationPageJs)

    # Fetch the latest notifications from Firebase
    FBconn = firebase.FirebaseApplication(firebase_url, None)
    notifications_data = FBconn.get('/notifications', None)
    # print(f'Fetched notifications data: {notifications_data}')  # Debug log to check fetched data

    if notifications_data:  # Check if notifications_data is not None or empty
        # Flatten the notifications data
        all_notifications = []
        for key, value in notifications_data.items():
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    if isinstance(sub_value, dict):
                        all_notifications.append(sub_value)
                    else:
                        all_notifications.append(value)
                        break
            else:
                all_notifications.append(value)

        # print(f'All notifications: {all_notifications}')  # Debug log to check flattened notifications

        # Sort the notifications by timestamp and get the latest 5
        sorted_notifications = sorted(all_notifications, key=lambda x: x['timestamp'], reverse=True)[:5]
        # print(f'Sorted notifications: {sorted_notifications}')  # Debug log to check sorted notifications

        # Prepare notifications for display
        notifications_for_display = [
            {
                "description": notification.get('message', notification.get('description')),
                "student_name": notification.get('student_name', f'Student {notification.get("student_id", "Unknown")}'),
                "timestamp": notification['timestamp']
            }
            for notification in sorted_notifications
        ]
        # print(f'Notifications for display: {notifications_for_display}')  # Debug log for display data

        # Store the notifications in localStorage for display in JavaScript
        display_html = f"""
        <script>
            console.log('Notifications for display:', {notifications_for_display});  // Debugging log
            localStorage.setItem('latestNotifications', JSON.stringify({notifications_for_display}));
            displayNotifications();  // Call the display function immediately
        </script>
        """
        display(HTML(display_html))
    else:
        print("No notifications found.")  # Debug log if no notifications found



output.register_callback('notebook.moveUpload', move_upload)
output.register_callback('notebook.moveGlossary', move_glossary)
output.register_callback('notebook.moveTeamStats', move_team_stats)
output.register_callback('notebook.moveNotifications', move_notifications)


def displayHtml(body, js):
    html_template = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>My HTML Page</title>
        <style>
            {css_content}
        </style>
    </head>
    <body>
        <div class="button-container">
          <button class="nav-button" id="btnIndex" onclick="loadPage('JSONUpload')">JSON Upload</button>
          <button class="nav-button" id="btnGlossary" onclick="loadPage('Glossary')">Glossary</button>
          <button class="nav-button" id="btnTeamGraph" onclick="loadPage('TeamStats')">Team Stats</button>
          <button class="nav-button" id="btnNotifications" onclick="loadPage('Notifications')">Notifications</button>
      </div>

      <div id="notificationModal" class="modal">
          <div class="modal-content">
            <span class="close">&times;</span>
            <p id="modalContent"></p>
          </div>
      </div>

        {body_content}
        <script>
            {js_content}
        </script>
        <script>
            {uploadjsonfunc}
        </script>
    </body>
    </html>
    """

    # Insert your existing CSS and JS variables
    html_content = html_template.format(
        css_content=style_css,
        js_content=js,
        body_content=body,
        uploadjsonfunc=uploadjson
    )

    # Display or save the generated HTML
    display(HTML(html_content))

displayHtml(uploadPage, uploadPageJs)

